In [ ]:
!pip install pinecone-client
!pip install python-dotenv
!pip install langchain
!pip install PyPDF
!pip install PyPDF2
!pip install OpenAI
!pip install tiktoken

In [13]:
import pinecone
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Pinecone

In [14]:
!cat > .env

^C


In [28]:
from dotenv import load_dotenv
import os
load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

OPENAI_API_KEY=os.getenv("OPENAI_API_KEY")

In [29]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = 'weirdfacts'
index = pc.Index(index_name)




In [30]:
from langchain.document_loaders import UnstructuredPDFLoader, OnlinePDFLoader, PyPDFLoader
loader = PyPDFLoader("/content/WEIRD_HYPOTHTETICAL_FACTS.pdf")

In [31]:
file_content = loader.load()

In [32]:
len(file_content)

1

In [33]:
print(file_content)

[Document(page_content='1.\nIn\na\nworld\nwhere\ngravity\nfluctuates\nunpredictably,\npeople\nmust\nwear\nspecialized\nboots\nequipped\nwith\nanti-gravity\ntechnology\nto\nremain\ngrounded.\n2.\nImagine\na\nsociety\nwhere\nemotions\nare\ntangible\nobjects\nthat\ncan\nbe\ntraded,\nbought,\nand\nsold\nin\na\nbustling\nmarketplace.\n3.\nPicture\na\nplanet\nwhere\nplants\ncommunicate\nthrough\nintricate\nmusical\nmelodies,\nforming\nharmonious\nsymphonies\nthat\necho\nthrough\nthe\nforests.\n4.\nConsider\na\nparallel\nuniverse\nwhere\ntime\nflows\nbackward,\ncausing\npeople\nto\nage\nin\nreverse\nand\nmemories\nto\nfade\ninto\nthe\nfuture.\n5.\nIn\na\ndimension\ngoverned\nby\ncolor,\nindividuals\npossess\nthe\nability\nto\nmanipulate\nhues\nto\nalter\nthe\nfabric\nof\nreality\nitself.\n6.\nEnvision\na\nrealm\nwhere\ndreams\nmanifest\nas\nphysical\nentities,\nwandering\nthe\nstreets\nlike\nethereal\nbeings\nseeking\npurpose.\n7.\nPicture\na\ncivilization\nwhere\nevery\ndecision\nis\ndetermi

In [34]:

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap  = 0,
    length_function = len,
)
book_texts = text_splitter.split_documents(file_content)
print (len(book_texts))
type(book_texts)



2


list

In [35]:
from langchain.embeddings.openai import OpenAIEmbeddings

In [36]:
embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

In [37]:
from langchain.vectorstores import Pinecone
book_docsearch = Pinecone.from_documents(book_texts, embeddings, index_name = index_name)

In [38]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)


In [40]:
query = "what happens in mirror dimension?"
docs = book_docsearch.similarity_search(query)



In [41]:
docs[0]

Document(page_content='universe,\nglowing\nwith\notherworldly\nwisdom.\n9.\nIn\na\nworld\nwhere\nmirrors\nserve\nas\ngateways\nto\nalternate\ndimensions,\nreflections\nbecome\nportals\nto\nparallel\nlives.\n10.\nConsider\na\nrealm\nwhere\nwhispers\nhold\nimmense\npower,\nshaping\nthe\ncourse\nof\nhistory\nwith\neach\nuttered\nword.', metadata={'page': 0.0, 'source': '/content/WEIRD_HYPOTHTETICAL_FACTS.pdf'})

In [42]:
from langchain.chains.question_answering import load_qa_chain


chain = load_qa_chain(llm, chain_type="stuff")
chain.run(input_documents=docs, question=query)




/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


' In the mirror dimension, reflections become portals to parallel lives.'